# Examining Political Donations

In [ ]:
from arcgis.gis import GIS
from IPython.display import display
import pandas as pd

In [ ]:
gis = GIS("https://slustl.maps.arcgis.com/", "gregbrunner_slugis")

### Access and Geocode Address Data

In [ ]:
item = gis.content.get("57894e2fb17b47b08c2608887a8e4897")
item

In [ ]:
voter_file = gis.content.get("57894e2fb17b47b08c2608887a8e4897")
csv_file = voter_file.download(save_path=r"c:\temp", file_name="address.csv" )
df = pd.read_csv(csv_file, encoding="ISO-8859-1")
df.head()

In [ ]:
geocoded_fc = gis.content.import_data(df=df)
geocoded_fc

In [ ]:
gc_map = gis.map("Maryland")
gc_map.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
                 'xmin': -8586268.28378512,
                 'ymin': 4733599.9383514235,
                 'xmax': -8511665.744178887,
                 'ymax': 4764174.749665453}
gc_map

In [ ]:
gc_map.add_layer(geocoded_fc)

### Access Census Data

In [ ]:
item = gis.content.get("2cb6e7dfc87047ccaa38f59d955d907b")
item

In [ ]:
sdf = item.layers[0].query("CNTY2010 = '24027'", as_df=True)
sdf.head()

In [ ]:
sdf.spatial.plot()

### Summarize Locations within Polygons

In [ ]:
from arcgis.features.summarize_data import summarize_within

In [ ]:
from datetime import datetime
result = summarize_within(sum_within_layer=sdf.spatial.to_feature_collection(), 
                          output_name="SummaryWithin_" + str(datetime.now().microsecond),
                          summary_layer=geocoded_fc, 
                          sum_shape=True, 
                          summary_fields=["Donate SUM"])

In [ ]:
result

In [ ]:
m4 = gis.map()
m4.extent = result.extent
m4.basemap = 'dark-gray'
m4

In [ ]:
m4.add_layer(result)

### Enrich Data to Locate More Information

- use `enrich` to load the Age data

In [ ]:
from arcgis.geoenrichment import *

In [ ]:
usa = Country.get('United States')
df = usa.data_collections
df.loc['Age']['analysisVariable'].unique()

In [ ]:
count_sdf = result.layers[0].query(as_df=True)
count_sdf.head()

In [ ]:
enrich_sdf = enrich(study_areas=count_sdf, data_collections=['Age'])
enrich_sdf.head()

### Find the Dollar Raised Per Person

#### Summarize the Total Population

In [ ]:
cols = [col for col in enrich_sdf.columns if (col.find("FEM") > -1 or col.find("MALE") > -1)]
enrich_sdf['TOTAL_POP'] = enrich_sdf[cols].sum(axis=1)

#### Clean up the Data

In [ ]:
enrich_sdf["SUM_Donate"] = (enrich_sdf["SUM_Donate"]
                            .replace('NaN', 0)
                            .astype(int))

#### Calculate Donation Dollars Per Person

In [ ]:
enrich_sdf["DOLLARPERPERSON"] = enrich_sdf['SUM_Donate'] / enrich_sdf['TOTAL_POP']
q = enrich_sdf["DOLLARPERPERSON"] > 0
target_tracts = enrich_sdf[q].copy().reset_index(drop=True)

In [ ]:
m3 = gis.map("Maryland")
m3.basemap = 'dark-gray'
m3

In [ ]:
m3.extent = target_tracts.spatial.full_extent
target_tracts.spatial.plot(map_widget=m3,
                           renderer_type='c',  # for class breaks renderer
                           method='esriClassifyNaturalBreaks',  # classification algorithm
                           class_count=10,  # choose the number of classes
                           col='DOLLARPERPERSON',  # numeric column to classify
                           cmap='prism_r',  # color map to pick colors from for each class
                           alpha=0.4)

### Generate Age Profile from Donations

- Use matplotlib to display data in a visual way

#### Define the Pyramid

In [ ]:
def age_pyramid(df):
    import warnings
    import seaborn as sns
    import matplotlib.pyplot as plt

    %matplotlib inline
    warnings.simplefilter(action='ignore', category=FutureWarning)
    pd.options.mode.chained_assignment = None 
    plt.style.use('ggplot')

    df = df[[x for x in df.columns if 'MALE' in x or 'FEM' in x]]
    sf = pd.DataFrame(df.sum())
    age = sf.index.str.extract('(\d+)').astype('int64')
    f = sf[sf.index.str.startswith('FEM')]
    m = sf[sf.index.str.startswith('MALE')]
    sf = sf.reset_index(drop = True)
    f = f.reset_index(drop = True)
    m = m.reset_index(drop = True)
    sf['age'] = age
    f["age"] = age
    m["age"] = age
    f = f.sort_values(by='age', ascending=False).set_index('age')
    m = m.sort_values(by='age', ascending=False).set_index('age')
    

    popdf = pd.concat([f, m], axis=1)
    popdf.columns = ['F', 'M']
    popdf['agelabel'] = popdf.index.map(str) + ' - ' + (popdf.index+4).map(str)
    popdf.M = -popdf.M
    
    sns.barplot(x="F", y="agelabel", color="#CC6699", label="Female", data=popdf, edgecolor='none')
    sns.barplot(x="M",  y="agelabel", color="#008AB8", label="Male",   data=popdf,  edgecolor='none')
    plt.ylabel('Age group')
    plt.xlabel('Number of people');
    return plt;

#### Generate the Age Pyramid for All Donations

In [ ]:
age_pyramid(target_tracts)

#### Generate the Age Pyramid for Top Grossing Areas

- The age population skews younger than the donation as a whole

In [ ]:
age_pyramid(target_tracts[target_tracts.DOLLARPERPERSON > 4].copy())